In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
CONTINUAL = False

def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env

config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "create_env_on_driver": True,"train_batch_size": 256,"batch_mode": "complete_episodes",
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000039/checkpoint-39'
    trainer.restore(checkpoint_path=model_path)

# models_dir = os.path.join('./' + PROJECT + '/models/')
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)
# text_name = models_dir + TRIAL + '.txt'
# with open(text_name, "w") as fp:
#     trainer.get_policy().model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
# png_name = models_dir + TRIAL + '.png'
# plot_model(trainer.get_policy().model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

2021-12-09 01:49:46,253	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2021-12-09 01:49:46,255	WARNING ppo.py:151 -- `train_batch_size` (256) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 256.
2021-12-09 01:49:46,255	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-12-09 01:49:46,256	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-12-09 01:49:47,283	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been 

In [ ]:
#def getkey(key):
    # return 111
#    return(bool(ctypes.windll.user32.GetAsyncKeyState(key) & 0x8000))
# Training & evaluation

record_mode = 1
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.reset()
    results = trainer.train()
    print(pretty_print(results))
    check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n----------------- Evaluation at steps:{steps} starting ! -----------------')
        print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        win = 0
        for i in range(NUM_EVAL*0):
            # print(f'\nEvaluation {i}:')
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            fig = plt.figure(1)
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100
            env_blue_pos = [0]
            env_red_pos = [0]
            env_mrm_pos = [0]
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(eval_env.WINNDOW_SIZE_lon,eval_env.WINDOW_SIZE_lat))

            while True:
                action_dict = {}
                for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))
                obs, rewards, dones, infos = eval_env.step(action_dict)
                env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
                env_blue_pos.append(env_blue_pos_temp)
                env_red_pos.append(env_red_pos_temp)
                env_mrm_pos.append(env_mrm_pos_temp)
                if step_num == 0:
                    del env_blue_pos[0]
                    del env_red_pos[0]
                    del env_mrm_pos[0]

                hist_blue_pos = np.vstack(env_blue_pos)
                hist_red_pos = np.vstack(env_red_pos)
                hist_mrm_pos = np.vstack(env_mrm_pos)
                plt.clf()
                render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
                render_env.rend_3d(eval_env,hist_red_pos,"r",1)
                render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
                fig.canvas.draw()
                plt.pause(.05)
                if record_mode == 0:
                    img = np.array(fig.canvas.renderer.buffer_rgba())
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    # cv2.imshow('test', img)
                    # cv2.waitKey(1)
                    # cv2.destroyAllWindows()
                    video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            if record_mode == 0:
                video.release()

ray.shutdown()


----------------- Training at steps:0 start! -----------------
blue_0DOWN
LOSE
blue_0 True True 200 -2.0 -1.0049999999999992
blue_1 True True 200 0.005 1.0000000000000007
blue_1DOWN
LOSE
blue_0 True True 85 0.005 0.42500000000000027
blue_1 True True 85 -2.0 -1.5799999999999996


2021-12-09 01:50:59,633	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 570
custom_metrics: {}
date: 2021-12-09_01-51-01
done: false
episode_len_mean: 142.5
episode_media: {}
episode_reward_max: -0.005000000000020437
episode_reward_mean: -0.58000000000001
episode_reward_min: -1.1549999999999994
episodes_this_iter: 2
episodes_total: 2
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 8.571272850036621
        entropy_coeff: 0.0
        kl: 0.021246980875730515
        model: {}
        policy_loss: 0.020580817013978958
        total_loss: 0.4572502672672272
        vf_explained_var: 0.13450315594673157
        vf_loss: 0.43242010474205017
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 8.505207061767578
        entropy_coeff: 0.0
        kl: 0.0

blue_1DOWN
LOSE
blue_0 True True 141 0.005 0.7050000000000005
blue_1 True True 141 -2.0 -1.2999999999999994
blue_0DOWN
LOSE
blue_0 True True 40 -2.0 -1.805
blue_1 True True 40 0.005 0.2000000000000001
blue_1DOWN
LOSE
blue_0 True True 165 0.005 0.8250000000000006
blue_1 True True 165 -2.0 -1.1799999999999993
agent_timesteps_total: 3062
custom_metrics: {}
date: 2021-12-09_01-55-26
done: false
episode_len_mean: 139.1818181818182
episode_media: {}
episode_reward_max: 1.819999999999971
episode_reward_mean: -0.5918181818181899
episode_reward_min: -1.605
episodes_this_iter: 3
episodes_total: 11
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 8.396687507629395
        entropy_coeff: 0.0
        kl: 0.015333007089793682
        model: {}
        policy_loss: 0.042596373707056046
        total_loss: 0.5205609

blue_0DOWN
LOSE
blue_0 True True 135 -2.0 -1.3299999999999996
blue_1 True True 135 0.005 0.6750000000000005
blue_1DOWN
LOSE
blue_0 True True 96 0.005 0.4800000000000003
blue_1 True True 96 -2.0 -1.5249999999999997
blue_0DOWN
LOSE
blue_0 True True 242 -1.998 -0.7720000000000036
blue_1 True True 242 0.005 1.2099999999999962
agent_timesteps_total: 5608
custom_metrics: {}
date: 2021-12-09_01-59-46
done: false
episode_len_mean: 140.2
episode_media: {}
episode_reward_max: 1.819999999999971
episode_reward_mean: -0.5901000000000086
episode_reward_min: -1.605
episodes_this_iter: 3
episodes_total: 20
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.30000001192092896
        cur_lr: 4.999999873689376e-05
        entropy: 8.437850952148438
        entropy_coeff: 0.0
        kl: 0.01497672125697136
        model: {}
        policy_loss: -0.2831854224205017
        total_loss: -0.07643

...

LOSE
blue_0 False False 600 -0.995 1.9999999999999583
blue_1 False False 600 -0.995 2.0349999999999615
agent_timesteps_total: 258214
custom_metrics: {}
date: 2021-12-09_07-08-49
done: false
episode_len_mean: 311.13
episode_media: {}
episode_reward_max: 4.175999999999897
episode_reward_mean: 1.1453099999999592
episode_reward_min: -1.645
episodes_this_iter: 1
episodes_total: 541
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 2.7368369102478027
        cur_lr: 4.999999873689376e-05
        entropy: 8.7867431640625
        entropy_coeff: 0.0
        kl: 0.0029009999707341194
        model: {}
        policy_loss: -0.38154640793800354
        total_loss: -0.3630065619945526
        vf_explained_var: -0.408381849527359
        vf_loss: 0.010600285604596138
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.8494508266448975
        cur_lr: 4.999999

blue_0DOWN
LOSE
blue_0 True True 182 -2.0 -1.0949999999999993
blue_1 True True 182 0.005 0.9100000000000007
LOSE
blue_0 False True 389 -0.995 0.9449999999999806
blue_1 False True 389 -0.993 1.1929999999999872
agent_timesteps_total: 260938
custom_metrics: {}
date: 2021-12-09_07-11-41
done: false
episode_len_mean: 313.86
episode_media: {}
episode_reward_max: 4.175999999999897
episode_reward_mean: 1.1757899999999588
episode_reward_min: -1.645
episodes_this_iter: 2
episodes_total: 545
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.3684184551239014
        cur_lr: 4.999999873689376e-05
        entropy: 7.207497596740723
        entropy_coeff: 0.0
        kl: 0.01744518056511879
        model: {}
        policy_loss: -0.12725695967674255
        total_loss: 0.2606959939002991
        vf_explained_var: -0.12363968044519424
        vf_loss: 0.36408063769340515
    blue_1:
     

LOSE
blue_0 False True 475 -0.995 1.3749999999999716
blue_1 False True 475 -0.996 1.6979999999999955
agent_timesteps_total: 264444
custom_metrics: {}
date: 2021-12-09_07-15-29
done: false
episode_len_mean: 313.69
episode_media: {}
episode_reward_max: 4.175999999999897
episode_reward_mean: 1.1781599999999586
episode_reward_min: -1.645
episodes_this_iter: 1
episodes_total: 551
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 3.0789413452148438
        cur_lr: 4.999999873689376e-05
        entropy: 7.688404560089111
        entropy_coeff: 0.0
        kl: 0.003574493806809187
        model: {}
        policy_loss: -0.44694870710372925
        total_loss: -0.4114289879798889
        vf_explained_var: -0.8193509578704834
        vf_loss: 0.024514008313417435
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.8494508266448975
        cur_lr: 4.999999

blue_0DOWN
LOSE
blue_0 True True 508 -2.0 0.5349999999999682
blue_1 True True 508 0.005 2.5269999999999695
agent_timesteps_total: 268732
custom_metrics: {}
date: 2021-12-09_07-19-51
done: false
episode_len_mean: 323.57
episode_media: {}
episode_reward_max: 4.175999999999897
episode_reward_mean: 1.2781799999999566
episode_reward_min: -1.645
episodes_this_iter: 1
episodes_total: 556
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.5394706726074219
        cur_lr: 4.999999873689376e-05
        entropy: 4.99015474319458
        entropy_coeff: 0.0
        kl: 0.008565478026866913
        model: {}
        policy_loss: -0.4983275532722473
        total_loss: -0.46636873483657837
        vf_explained_var: 0.26920872926712036
        vf_loss: 0.018772434443235397
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.8494508266448975
        cur_lr: 4.9

blue_1DOWN
LOSE
blue_0 True True 123 0.005 0.6150000000000004
blue_1 True True 123 -2.0 -1.3899999999999997
LOSE
blue_0 False True 419 -0.995 1.0949999999999775
blue_1 False True 419 -0.993 1.4449999999999932
agent_timesteps_total: 271574
custom_metrics: {}
date: 2021-12-09_07-22-37
done: false
episode_len_mean: 326.64
episode_media: {}
episode_reward_max: 4.175999999999897
episode_reward_mean: 1.3113999999999564
episode_reward_min: -1.645
episodes_this_iter: 2
episodes_total: 560
experiment_id: 3258f0277f3a4fa98a0ca2d0d9f5979d
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 2.309206008911133
        cur_lr: 4.999999873689376e-05
        entropy: 7.618456840515137
        entropy_coeff: 0.0
        kl: 0.009162690490484238
        model: {}
        policy_loss: -0.17531463503837585
        total_loss: -0.06399530917406082
        vf_explained_var: -0.35879993438720703
        vf_loss: 0.0901607945561409
    blue_1:
    